In [50]:
class Layer:
  
    def __init__(object):
        object.ip = None
        object.op = None

    def forward(object, data_ip):
        raise NotImplementedError

    def backward(object, error_op, alpha):
        raise NotImplementedError

##**Implementing Linear Layer**


In [51]:
import numpy as np

class LinearLayer(Layer):

    def __init__(object, len_ip, len_op):
        object.wghts = np.random.rand(len_ip, len_op)
        object.bs = np.random.rand(1, len_op)

    def forward(object, ip_data):
        object.ip = ip_data
        temp = np.dot(object.ip, object.wghts)
        object.op=temp+object.bs
        return object.op

    def backward(object, op_error, alpha):
        ip_error = np.dot(op_error, object.wghts.T)
        wghts_error = np.dot(object.ip.T, op_error)
        object.wghts =object.wghts- (alpha * wghts_error)
        object.bs =object.bs- (alpha * op_error)
        return ip_error

##**Tangent Activation Class**


In [52]:
class TangentActivation(Layer):
    
    def forward(object, ip_data):
        object.ip = ip_data
        object.op = np.tanh(object.ip)
        return object.op

    def backward(object, op_error, alpha):
        return (1-np.tanh(object.ip)**2 )* op_error

##**Sigmoid Activation Class**

In [53]:
class SigmoidActivation(Layer):

    def forward(object, ip_data):
        object.ip = ip_data
        object.op =  (1/(1 + np.exp(-object.ip)))
        return object.op

    def backward(object, op_error, alpha):
        return (np.exp(-object.ip)/(1+np.exp(-object.ip)**2))* op_error

##**SoftMax Activation Class**

In [54]:
class SoftMaxActivation(Layer):

    def forward(object, ip_data):
        object.ip = ip_data
        object.op =  (np.exp(ip_data - np.max(ip_data)) / np.exp(ip_data - np.max(ip_data)).sum())
        return object.op

    def backward(object, op_error, alpha):
        input_error = np.zeros(op_error.shape)
        out = np.tile(object.op.T, len(object.ip))
        return object.op * np.dot(op_error, np.identity(len(object.ip)) - out)

##**Cross Entropy Loss and Accuracy Calculator**

In [55]:
import numpy as np

def cross_entropy_loss(projected,original):
    return -original * np.log(projected)

def cross_entropy_loss_grad(projected,original):
        return original - projected

def find_accuracy(projected,original):
    count=0
    for i in range(len(projected)):
      if projected[i]==original[i]:
        count+=1    
    return count/len(projected)*100


##**Implementing Sequential Class**

In [59]:
class Sequential:
    def __init__(object):
        object.lyrs = []

    def appendLayer(object, lyr):
        object.lyrs.append(lyr)

    #Predicting Using Forward Propagation
    def findOutput(object, ip_data):
        result = []
        itr=0
        while itr < len(ip_data):
            op = ip_data[itr]
            for lyr in object.lyrs:
                op = lyr.forward(op)
            result.append(op)
            itr=itr+1
        return result

    #Training Neural Network
    def trainnetwork(object, tr_data, tr_y, noofiterations, alpha):
        itr=0
        jtr=0
        while itr < noofiterations:
            itr=itr+1        
            for jtr in range(len(tr_data)):
                op = tr_data[jtr]
                for lyr in object.lyrs:
                    op = lyr.forward(op)
                #finding error and propagating at each layer
                error = cross_entropy_loss_grad(tr_y[jtr], op)
                for lyr in reversed(object.lyrs):
                    error = lyr.backward(error, alpha)

            

##**Building Neural Network For XOR Function using Sigmoid Function**

In [67]:
import numpy as np

# Training Data 
data_train_x = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
data_train_y = np.array([[[0]], [[1]], [[1]], [[0]]])

#Using Sigmoid function

#Building Network
nnworkSigm = Sequential()
nnworkSigm.appendLayer(LinearLayer(2, 3))
nnworkSigm.appendLayer(SigmoidActivation())
nnworkSigm.appendLayer(LinearLayer(3, 2))
nnworkSigm.appendLayer(SigmoidActivation())

#Fitting Network
nnworkSigm.trainnetwork(data_train_x, data_train_y, 2000,0.01)

#Predicting
out = nnworkSigm.findOutput(data_train_x)
print(out)

#Finding Output Number
outLables=[]
for i in range(3):
  outLables.append(np.argmax(out[i]))

#Finding Accuracy
print("Accuracy of Network with Sigmoid Function Is ",find_accuracy(np.array(outLables),y_train[:]),"%")


[array([[0.50903382, 0.49528301]]), array([[0.50709438, 0.49556121]]), array([[0.49471008, 0.50492721]]), array([[0.49289442, 0.50502134]])]
Accuracy of Network with Sigmoid Function Is  66.66666666666666 %


##**Building Neural Network For XOR Function using Tanh Function**

In [66]:
##Using Tan function

#Building Network
nnworkTan = Sequential()
nnworkTan.appendLayer(LinearLayer(2, 2))
nnworkTan.appendLayer(SigmoidActivation())
nnworkTan.appendLayer(LinearLayer(2, 1))
nnworkTan.appendLayer(SigmoidActivation())

#Fitting Network
nnworkTan.trainnetwork(x_train, y_train, 2000,0.01)

#Predicting
out = nnworkTan.findOutput(x_train)

#Finding Output Number
outLables=[]
for i in range(3):
  outLables.append(np.argmax(out[i]))

print(outLables)
#Finding Accuracy
print("Accuracy of Network with Tanh Function Is ",find_accuracy(np.array(outLables),y_train[:]),"%")

[0, 0, 0]
Accuracy of Network with Tanh Function Is  33.33333333333333 %


##**Building Neural Network For MNIST Dataset**
##Importing Dataset


In [69]:
import numpy as np
from keras.utils import to_categorical
from keras.datasets import mnist

#Nodes in the layers
inp_layer=784;
hidden1=128;
hidden2=64;
oup_layer=10;

#Loading Dataset
(data_train_x, data_train_y), (data_test_x, data_test_y) = mnist.load_data()

#Normalizing and reshaping the training data
data_train_x = data_train_x.reshape(data_train_x.shape[0], 1, inp_layer)
data_train_x = (data_train_x/255).astype('float32')

#Normalizing and reshaping the test data
data_test_x = data_test_x.reshape(data_test_x.shape[0], 1, inp_layer)
data_test_x = (data_test_x/255).astype('float32')


#One Hard Encoding Target data
arr = []
for i in range(len(data_train_y)):
  arr.append(to_categorical(data_train_y[i], num_classes=10))
data_train_y = np.array(arr)


##**Using Softmax as Activation Function**

In [72]:
#Adding Layers
mnistNetwork = Sequential()
mnistNetwork.appendLayer(LinearLayer(inp_layer,hidden1))
mnistNetwork.appendLayer(SoftMaxActivation())
mnistNetwork.appendLayer(LinearLayer(hidden1,hidden2))
mnistNetwork.appendLayer(SoftMaxActivation())
mnistNetwork.appendLayer(LinearLayer(hidden2,oup_layer))
mnistNetwork.appendLayer(SoftMaxActivation())

n=10000
#Fitting Network
mnistNetwork.trainnetwork(data_train_x[:n], data_train_y[:n], 5, 0.1)

#Finding Output Number
outLables=[]
out = mnistNetwork.findOutput(data_test_x[:n])
for i in range(n):
  outLables.append(np.argmax(out[i]))

#Finding Accuracy
print("Accuracy of Network  Is ",find_accuracy(np.array(outLables),data_test_y[:n]),"%")

Accuracy of Network  Is  9.82 %


##**Using Sigmoid as Activation Function**

In [73]:
#Adding Layers
mnistNetwork = Sequential()
mnistNetwork.appendLayer(LinearLayer(inp_layer,hidden1))
mnistNetwork.appendLayer(SigmoidActivation())
mnistNetwork.appendLayer(LinearLayer(hidden1,hidden2))
mnistNetwork.appendLayer(SigmoidActivation())
mnistNetwork.appendLayer(LinearLayer(hidden2,oup_layer))
mnistNetwork.appendLayer(SigmoidActivation())

n=10000
#Fitting Network
mnistNetwork.trainnetwork(data_train_x[:n], data_train_y[:n], 5, 0.1)

#Finding Output Number
outLables=[]
out = mnistNetwork.findOutput(data_test_x[:n])
for i in range(n):
  outLables.append(np.argmax(out[i]))

#Finding Accuracy
print("Accuracy of Network  Is ",find_accuracy(np.array(outLables),data_test_y[:n]),"%")

Accuracy of Network  Is  10.09 %


##**Using Tanh as Activation Function**

In [74]:
#Adding Layers
mnistNetwork = Sequential()
mnistNetwork.appendLayer(LinearLayer(inp_layer,hidden1))
mnistNetwork.appendLayer(TangentActivation())
mnistNetwork.appendLayer(LinearLayer(hidden1,hidden2))
mnistNetwork.appendLayer(TangentActivation())
mnistNetwork.appendLayer(LinearLayer(hidden2,oup_layer))
mnistNetwork.appendLayer(TangentActivation())

n=10000
#Fitting Network
mnistNetwork.trainnetwork(data_train_x[:n], data_train_y[:n], 5, 0.1)

#Finding Output Number
outLables=[]
out = mnistNetwork.findOutput(data_test_x[:n])
for i in range(n):
  outLables.append(np.argmax(out[i]))

#Finding Accuracy
print("Accuracy of Network  Is ",find_accuracy(np.array(outLables),data_test_y[:n]),"%")

Accuracy of Network  Is  9.8 %
